In [1]:
import os
import sys
current_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_dir, ".."))
code_dir = os.path.join(working_dir, "code")
data_dir = os.path.join(working_dir, "data")
sys.path.append(code_dir)

from molecules import get_molecule_N2

In [ ]:
import numpy as np
from pyscf import gto, scf, fci, mcscf
import pennylane as qml

bond_distance_list = [0.5, 0.75, 1., 1.25, 1.5, 1.75, 2., 2.25, 2.5, 2.75, 3., 3.25, 3.5, 3.75, 4.]
print(len(bond_distance_list), "bond distances")
energy_dict_hf = {}
energy_dict_fci = {}
energy_dict_afci = {}

for d in bond_distance_list:
    print("【Bond distance】", d, "Angstrom")
    symbols, geometry, electrons, orbitals, charge = get_molecule_N2(d)

    mol = gto.Mole()
    mol.atom = list(zip(symbols, geometry))
    mol.basis = 'STO-3G'
    mol.unit = 'bohr'
    mol.spin = 0
    mol.charge = charge
    mol.build()

    # RHF
    mf = scf.RHF(mol)
    mf.conv_tol = 1e-16
    hf_energy = mf.kernel()
    print(f"HF Energy: {hf_energy:.8f} Ha")

    # Note: mf.mo_coeff accuracy might be sqrt(1e-16)=1e-8
    cisolver = fci.FCI(mol, mf.mo_coeff)
    cisolver.conv_tol = 1e-20
    cisolver.conv_tol_residual = 1e-10
    cisolver.max_cycle = 20000
    fci_energy, fci_vector = cisolver.kernel()

    # Unrestricted CASCI
    ncas = 6         # Active space orbitals
    nelecas = 6      # Active space electrons
    my_casci = mcscf.UCASCI(mf, ncas, nelecas)

    mol_dhf = qml.qchem.Molecule(symbols, geometry, basis_name="sto-3g")
    if hasattr(mol_dhf.coordinates, "requires_grad"):
        mol_dhf.coordinates.requires_grad = False
    eigvals, C_dhf, *_ = qml.qchem.scf(mol_dhf)()   # AO→MO
    my_casci.mo_coeff = (C_dhf.copy(), C_dhf.copy())  # set CASCI MO coefficients from PennyLane

    #my_casci.verbose = 4
    casci_energy = my_casci.kernel()[0]
    #ci = my_casci.ci
    #print("CASCI Energy:", casci_energy)
    #print("S^2 = ", my_casci.fcisolver.spin_square(ci, my_casci.ncas, my_casci.nelecas))

    energy_dict_hf[d] = hf_energy
    energy_dict_fci[d] = casci_energy
    print("HF Energy: ", hf_energy, "Ha")
    print("FCI energy:", casci_energy, "Ha")
